In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
inputs= pd.read_csv("./dataset/inputs.csv",names=['txid','prev_tx_id','prev_tx_pos'])
outputs = pd.read_csv("./dataset/outputs.csv",names=['txid','position','address_id','amount','script_type'])
outputs=outputs.astype({
    'script_type':'uint8',
})
transactions = pd.read_csv("./dataset/transactions.csv",names=['timestamp','block_id','txid','is_coinbase','fee'])
transactions= transactions.astype({
    'is_coinbase':'bool',
})
map=pd.read_csv("./dataset/mapAddr2Ids8708820.csv",names=['hash','address_id'])
map.set_index('address_id')

# 2.1 Analisi generali dei dati della blockchain

## distribuzione transazioni per blocco
distribuzione del numero di transazioni per blocco (occupazione del blocco), nell’intero
periodo temporale considerato

In [ ]:
tx_distribution=transactions.groupby(['block_id']).count()[['timestamp']]
res=pd.DataFrame(columns=['blocks'])
res.index.name='value'
for (i,g) in tx_distribution.groupby(['timestamp']):
    res.loc[i]=len(g)
plt.figure(figsize=(14,5))
plt.scatter(np.log(res.index),res['blocks'],s=1)
plt.xlabel('numero di transazioni')
plt.ylabel('blocchi')

#TODO : fix display of data


## evoluzione dell'occupazione dei blocchi nel tempo
evoluzione dell'occupazione dei blocchi nel tempo, considerando intervalli temporali di due
mesi.(media dell'occupazione di blocchi su archi temporali di 2 mesi).

In [ ]:
from itertools import pairwise

transactions['date']=pd.to_datetime(transactions['timestamp'],unit='s')
month_transactions=[]
month_group=transactions.groupby([transactions['date'].dt.year,transactions['date'].dt.month])

average_tx=pd.DataFrame(columns=['average_tx'])

for (k1,k2) in pairwise(month_group.groups.keys()):
    months_df=pd.concat([month_group.get_group(k1),month_group.get_group(k2)])
    y,m1=k1
    _,m2=k2
    average_tx.loc[str(y)+'-'+str(m1)+'/'+str(m2)]=months_df.groupby(['block_id']).count()['timestamp'].mean()
average_tx.plot(kind='bar',figsize=(14,5))
# TODO: fix display of data

## ammontare degli ultimi UTXO rimanenti
ammontare totale degli UTXO al momento dell’ultima transazione registrata nella
blockchain considerata.

In [ ]:
spent_utxos=outputs.merge(inputs,how='inner',left_on=['txid','position'],right_on=['prev_tx_id','prev_tx_pos'])
unspent_utxos_amount=sum(outputs.amount)-sum(spent_utxos.amount)
unspent_utxos_amount

## durata media di un UTXO prima di essere speso
distribuzione degli intervalli di tempo che intercorrono tra la transazione che genera un
valore in output (UTXO) e quella che lo consuma, per gli output spesi nel periodo
considerato.

In [ ]:
#a=inputs.merge(transactions[['txid','timestamp']],left_on='prev_tx_id',right_on='txid')
#a=a.merge(transactions[['txid','timestamp']],left_on='txid_x',right_on='txid')[['prev_tx_id','timestamp_x','prev_tx_pos','txid','timestamp_y']]

created_tx_ts=inputs.merge(transactions[['txid','timestamp']],left_on='prev_tx_id',right_on='txid')

spent_tx_ts=inputs.merge(transactions[['txid','timestamp']],left_on='txid',right_on='txid')

average_utxo_dt=pd.DataFrame()

average_utxo_dt['deltatime']=pd.to_timedelta(spent_tx_ts['timestamp']-created_tx_ts['timestamp'],unit='s')
average_utxo_dt=average_utxo_dt.groupby('deltatime',group_keys=True).apply(lambda x: len(x))
# TODO: finish data display

## analisi a piacere


## 

# 2.2 clusterizzazione degli indirizzi di bitcoin


In [ ]:
import networkx as nx
"""
considerare le transazioni che hanno un numero di input
maggiore di 1 e associare ad uno stesso utente tutti gli indirizzi utilizzati negli input della stessa
transazione.
"""
#associo ad ogni input l'output speso per  ottenere l'indirizzo corrispondente
tx_addresses=inputs.merge(outputs,how='inner',left_on=['prev_tx_id','prev_tx_pos'],right_on=['txid','position'])[['prev_tx_id','address_id']]
#tx_addresses=tx_addresses.merge(map)
tx_addresses=tx_addresses.groupby(['prev_tx_id'])
print('done 1')


In [ ]:
g=nx.Graph()

for k,group in tx_addresses:
    if(len(group)<=1):
        pass
    a=group['address_id']
    u,*v=a.to_list()
    for _v in v:
        g.add_edge(map.loc[u].address_id,map.loc[_v].address_id)
print('done 2')
nx.weakly_connected_components(g) 